In [1]:
import sys
import io
import json
import os
import random
import numpy as np
import torch
import pandas as pd
import csv
from textwrap import dedent
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)

2025-02-10 08:29:34.968267: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model_name = "path-to-model"
#model_name = "meta-llama/Llama-3.1-8B-Instruct"

In [3]:
device = torch.device("cuda")

In [4]:
# Load latest checkpoint, replace 166 with actual checkpoint number

NEW_MODEL="./events/checkpoint-166/"

tokenizer = AutoTokenizer.from_pretrained(NEW_MODEL)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map='auto',
)

model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=8)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Embedding(128264, 4096)

In [5]:
# Use Huggingface pipeline for prediction

pipe = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    return_full_text=False,
)

In [6]:
# PEFT requires to separately load the tuned layers and re-merge

from peft import PeftModel

model = PeftModel.from_pretrained(model, model_id='./events/checkpoint-166/')
model = model.merge_and_unload()

In [9]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files={'train': 'train.json', 'validation': 'val.json', 'test': 'test.json'}
)

In [13]:
def create_test_prompt(row):
    prompt = dedent(
        f"""
        {row['annotation']}
        """
    )
    messages = [
        {"role": "system",
         "content": "Provide an event label for the following text snippet! Do not output anything else!!!"},
        {"role": "user", "content": prompt},
    ]

    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

contexts = dataset['test']

In [15]:
predictions = []
for context in contexts:
    print("###################################")
    print(context['annotation'])
    print("------")
    prompt = create_test_prompt(context)
    result = pipe(prompt)[0]['generated_text']
    print(result)
    predictions.append(result)
    print("###################################")
    print('\n\n')

###################################
Johanna nahm das Blatt
------
non_event
###################################



###################################
Dieses Sich-in-Szene-setzen
------
non_event
###################################



###################################
wenn ich solch' steifleinenes Wort brauchen darf
------
non_event
###################################



###################################
und hätte, links Basedow und rechts Pestalozzi eigentlich nach Schnepfenthal oder Bunzlau hingepaßt
------
non_event
###################################



###################################
und Landgerichtsrat Gizicki sagte dann wohl zu Wüllersdorf
------
non_event
###################################



###################################
die, nach Frauenart, nicht ganz abgeneigt war, die ganze Sache als einen interessanten Fall anzusehen
------
non_event
###################################



###################################
und Effi hätte sich in die Dünen werfen
------
stat

In [17]:
from sklearn.metrics import accuracy_score, f1_score
f1_score(predictions, dataset["test"]["tag"], average="macro")

0.2412757585171378